In [2]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patheffects as pe
import seaborn as sns
from pathlib import Path
from dataclasses import dataclass, field
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import FuncFormatter
from matplotlib_scalebar.scalebar import ScaleBar
from datetime import datetime
import warnings
import unicodedata
import re
import math

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
warnings.filterwarnings("ignore")

@dataclass
class HydroConfig:
    # Caminhos relativos para garantir portabilidade
    BASE_DIR: Path = Path("..")
    PATH_MUNICIPIOS: Path = BASE_DIR / "inputs" / "BR_Municipios_2024" / "BR_Municipios_2024.shp"
    PATH_ESTADOS: Path = BASE_DIR / "inputs" / "BR_UF_2024" / "BR_UF_2024.shp"
    PATH_OUTPUT: Path = BASE_DIR / "output"
    
    NOME_MUNICIPIO: str = 'Sobradinho'
    SIGLA_UF: str = 'BA'
    ASSET_ID: str = "WWF/HydroATLAS/v1/Basins/level10"
    
    CRS_LATLON: str = 'EPSG:4674'
    CRS_METRIC: str = 'EPSG:3857'
    
    COLUMNS_MAP: dict = field(default_factory=lambda: {
        'HYBAS_ID': 'ID_Global',
        'SUB_AREA': 'Area_km2',
        'dis_m3_pyr': 'Vazao_m3s',
        'pre_mm_syr': 'Precipitacao_mm',
        'for_pc_sse': 'Floresta_pct'
    })

class GeoUtils:
    @staticmethod
    def normalize_string(text: str) -> str:
        if not isinstance(text, str): return ""
        return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8').lower().strip()

    @staticmethod
    def sanitize_filename(nome: str) -> str:
        return re.sub(r'[^\w\s-]', '', GeoUtils.normalize_string(nome)).strip().replace(' ', '_')

class HydroService:
    def __init__(self, config: HydroConfig):
        self.cfg = config
        self._authenticate()

    def _authenticate(self):
        try:
            ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
        except:
            ee.Authenticate()
            ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

    def get_basins_data(self, region_ee):
        basins = ee.FeatureCollection(self.cfg.ASSET_ID)
        local_basins = basins.filterBounds(region_ee.geometry())
        
        if local_basins.size().getInfo() == 0:
            return gpd.GeoDataFrame()
        
        cols = list(self.cfg.COLUMNS_MAP.keys())
        gdf = geemap.ee_to_gdf(local_basins.select(cols))
        
        gdf = gdf.rename(columns=self.cfg.COLUMNS_MAP)
        gdf['geometry'] = gdf.geometry.buffer(0)
        if gdf.crs is None:
            gdf.set_crs("EPSG:4326", inplace=True)
            
        gdf = gdf.sort_values(by='Area_km2', ascending=False).reset_index(drop=True)
        gdf['Short_ID'] = gdf.index + 1
        
        return gdf

class CartoRenderer:
    def __init__(self, config: HydroConfig):
        self.cfg = config

    def _styled_box(self, ax, title):
        ax.set_xticks([])
        ax.set_yticks([])
        for spine in ax.spines.values():
            spine.set_edgecolor('#333333')
            spine.set_linewidth(1)
        ax.set_title(title, fontsize=9, weight='bold', pad=6, backgroundcolor='#e0e0e0', color='black')

    def _setup_grid_lines(self, ax, gdf_latlon):
        if gdf_latlon.empty: return
        minx, miny, maxx, maxy = gdf_latlon.total_bounds
        x_target = np.linspace(minx, maxx, 4)
        y_target = np.linspace(miny, maxy, 4)
        
        pts_x = gpd.points_from_xy(x_target, [miny]*4, crs=self.cfg.CRS_LATLON).to_crs(self.cfg.CRS_METRIC)
        pts_y = gpd.points_from_xy([minx]*4, y_target, crs=self.cfg.CRS_LATLON).to_crs(self.cfg.CRS_METRIC)
        
        ax.set_xticks(pts_x.x)
        ax.set_yticks(pts_y.y)
        ax.xaxis.set_major_formatter(FuncFormatter(lambda x, p: f"{x_target[p]:.2f}°" if p < len(x_target) else ""))
        ax.yaxis.set_major_formatter(FuncFormatter(lambda y, p: f"{y_target[p]:.2f}°" if p < len(y_target) else ""))
        ax.tick_params(axis='both', which='major', labelsize=7, direction='in', pad=4)
        ax.grid(True, linestyle='--', alpha=0.3, color='#444444', linewidth=0.5, zorder=1)

    def _draw_minimap(self, ax, mun_gdf):
        self._styled_box(ax, f"LOCALIZAÇÃO - {self.cfg.SIGLA_UF}")
        try:
            if self.cfg.PATH_ESTADOS.exists():
                states = gpd.read_file(self.cfg.PATH_ESTADOS)
                state = states[states['SIGLA_UF'] == self.cfg.SIGLA_UF.upper()]
                if not state.empty:
                    state.to_crs(self.cfg.CRS_METRIC).plot(ax=ax, facecolor='#f4f4f4', edgecolor='#555555', linewidth=0.5)
                    mun_gdf.to_crs(self.cfg.CRS_METRIC).plot(ax=ax, facecolor='#d62728', edgecolor='none', zorder=2)
                else:
                    ax.text(0.5, 0.5, "UF N/A", ha='center', fontsize=8)
            else:
                ax.text(0.5, 0.5, "Shape UF ausente", ha='center', fontsize=8)
        except:
            ax.text(0.5, 0.5, "Erro Minimapa", ha='center', fontsize=8)

    def _draw_stats_box(self, ax, gdf, col, unit):
        self._styled_box(ax, "ESTATÍSTICAS")
        if gdf.empty:
            ax.text(0.5, 0.5, "Sem Dados", ha='center')
            return
        stats = gdf[col].describe()
        txt = (
            f"Média: {stats['mean']:,.2f} {unit}\n"
            f"Mínimo: {stats['min']:,.2f} {unit}\n"
            f"Máximo: {stats['max']:,.2f} {unit}\n"
            f"Desvio Padrão: {stats['std']:,.2f}\n"
            f"Total de Bacias: {int(stats['count'])}"
        )
        ax.text(0.5, 0.5, txt, ha='center', va='center', fontsize=9, linespacing=1.8, family='monospace')

    def _draw_info_box(self, ax):
        self._styled_box(ax, "") 
        ax.text(0.5, 0.95, "METADADOS", ha='center', va='top', fontsize=9, weight='bold', transform=ax.transAxes)
        txt = (
            f"Município: {self.cfg.NOME_MUNICIPIO}\n"
            f"Fonte: WWF HydroATLAS v1\n"
            f"Proc: GEE API & Python\n"
            f"Data: {datetime.now().strftime('%d/%m/%Y')}\n"
            f"CRS: SIRGAS 2000 / Web Mercator"
        )
        ax.text(0.5, 0.75, txt, ha='center', va='top', fontsize=8, linespacing=1.6, transform=ax.transAxes)

    def generate_map(self, basins_gdf, mun_gdf, column, title_prefix, cmap, unit_label):
        fig = plt.figure(figsize=(11.69, 8.27))
        gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1], wspace=0.1, figure=fig)
        
        ax_map = fig.add_subplot(gs[0])
        basins_proj = basins_gdf.to_crs(self.cfg.CRS_METRIC)
        mun_proj = mun_gdf.to_crs(self.cfg.CRS_METRIC)
        
        minx, miny, maxx, maxy = basins_proj.total_bounds
        margin = 5000 
        ax_map.set_xlim(minx - margin, maxx + margin)
        ax_map.set_ylim(miny - margin, maxy + margin)
        
        basins_proj.plot(column=column, ax=ax_map, cmap=cmap, alpha=0.9, edgecolor='#444444', linewidth=0.3, zorder=2)
        mun_proj.plot(ax=ax_map, facecolor='none', edgecolor='black', linewidth=1.5, linestyle='--', zorder=5)
        
        for _, row in basins_proj.iterrows():
            pt = row.geometry.representative_point()
            txt = ax_map.text(pt.x, pt.y, str(row['Short_ID']), fontsize=7, ha='center', va='center', weight='bold', color='white', zorder=6)
            txt.set_path_effects([pe.withStroke(linewidth=1.5, foreground='black')])

        self._setup_grid_lines(ax_map, mun_gdf.to_crs(self.cfg.CRS_LATLON))
        ax_map.add_artist(ScaleBar(1, units='m', location='lower left', box_alpha=0.7, font_properties={'family': 'serif', 'size': 8}))
        
        ax_map.annotate('N', xy=(0.95, 0.95), xytext=(0.95, 0.88), 
                        arrowprops=dict(facecolor='black', width=4, headwidth=10),
                        ha='center', va='center', fontsize=12, weight='bold', xycoords='axes fraction', zorder=10)
        
        ax_map.set_title(f"{title_prefix.upper()}\n{self.cfg.NOME_MUNICIPIO} - {self.cfg.SIGLA_UF}", fontsize=14, weight='bold', pad=15)

        gs_side = gridspec.GridSpecFromSubplotSpec(4, 1, subplot_spec=gs[1], height_ratios=[0.25, 0.15, 0.25, 0.35], hspace=0.4)
        self._draw_minimap(fig.add_subplot(gs_side[0]), mun_gdf)
        
        ax_leg = fig.add_subplot(gs_side[1])
        self._styled_box(ax_leg, "LEGENDA")
        if not basins_gdf.empty:
            vmin, vmax = basins_gdf[column].min(), basins_gdf[column].max()
            cbar = plt.colorbar(ScalarMappable(norm=Normalize(vmin, vmax), cmap=cmap), cax=ax_leg, orientation='horizontal', fraction=0.5, pad=0.1)
            cbar.ax.tick_params(labelsize=8)
            cbar.set_label(unit_label, fontsize=8, weight='bold')
        
        self._draw_info_box(fig.add_subplot(gs_side[2]))
        self._draw_stats_box(fig.add_subplot(gs_side[3]), basins_gdf, column, unit_label.split(' ')[0])
        return fig

    def generate_charts_page(self, df):
        fig = plt.figure(figsize=(11.69, 8.27))
        fig.suptitle(f"ANÁLISE ESTATÍSTICA - {self.cfg.NOME_MUNICIPIO}", fontsize=16, weight='bold', y=0.95)
        gs = gridspec.GridSpec(2, 2, height_ratios=[1, 1], hspace=0.3, wspace=0.3)
        vars = [('Vazao_m3s', 'Vazão (m³/s)', 'skyblue'), ('Precipitacao_mm', 'Precipitação (mm)', 'steelblue')]
        
        for i, (col, label, color) in enumerate(vars):
            ax_hist = fig.add_subplot(gs[i, 0])
            sns.histplot(df[col], kde=True, ax=ax_hist, color=color, edgecolor='black', linewidth=0.5)
            ax_hist.set_title(f"Distribuição: {label}", fontsize=10, weight='bold')
            ax_hist.set_xlabel(label, fontsize=9); ax_hist.set_ylabel("Frequência", fontsize=9)
            ax_hist.grid(True, alpha=0.2)
            
            ax_box = fig.add_subplot(gs[i, 1])
            sns.boxplot(x=df[col], ax=ax_box, color=color, width=0.4, linewidth=0.8)
            ax_box.set_title(f"Boxplot: {label}", fontsize=10, weight='bold')
            ax_box.set_xlabel(label, fontsize=9)
            ax_box.grid(True, alpha=0.2)
        return fig

    def generate_table_pages(self, basins_gdf):
        figs = []
        df_base = basins_gdf[['Short_ID', 'ID_Global', 'Area_km2', 'Vazao_m3s', 'Precipitacao_mm', 'Floresta_pct']].copy()
        df_base.columns = ['Ref.', 'HydroATLAS ID', 'Área (km²)', 'Vazão (m³/s)', 'Precip. (mm)', 'Floresta (%)']
        
        df_base['Área (km²)'] = df_base['Área (km²)'].apply(lambda x: f"{x:,.2f}")
        df_base['Vazão (m³/s)'] = df_base['Vazão (m³/s)'].apply(lambda x: f"{x:,.2f}")
        df_base['Floresta (%)'] = df_base['Floresta (%)'].apply(lambda x: f"{x:.1f}")

        rows_per_page = 25
        num_pages = math.ceil(len(df_base) / rows_per_page)

        for i in range(num_pages):
            fig, ax = plt.subplots(figsize=(11.69, 8.27))
            ax.axis('off')
            ax.set_title(f"TABELA DE DADOS ({i+1}/{num_pages})", weight='bold', fontsize=12, pad=10)
            
            df_page = df_base.iloc[i*rows_per_page : (i+1)*rows_per_page]
            table = ax.table(cellText=df_page.values, colLabels=df_page.columns, loc='center', cellLoc='center', colColours=['#eeeeee']*6)
            table.auto_set_font_size(False); table.set_fontsize(9); table.scale(1, 1.3)
            figs.append(fig)
        return figs

class HydroPipeline:
    def __init__(self, config: HydroConfig):
        self.cfg = config
        self.service = HydroService(config)
        self.renderer = CartoRenderer(config)

    def run(self):
        try:
            print(f"--- INICIANDO: {self.cfg.NOME_MUNICIPIO} ---")
            
            full_gdf = gpd.read_file(self.cfg.PATH_MUNICIPIOS, encoding='utf-8')
            full_gdf['norm_nm'] = full_gdf['NM_MUN'].apply(GeoUtils.normalize_string)
            
            aoi = full_gdf[(full_gdf['norm_nm'] == GeoUtils.normalize_string(self.cfg.NOME_MUNICIPIO)) & 
                           (full_gdf['SIGLA_UF'] == self.cfg.SIGLA_UF.upper())].dissolve()
            
            if aoi.empty: raise ValueError(f"Município {self.cfg.NOME_MUNICIPIO} não encontrado.")
            
            print("Consultando Earth Engine...")
            ee_obj = geemap.geopandas_to_ee(aoi) 
            basins = self.service.get_basins_data(ee_obj)
            
            if basins.empty: raise ValueError("Nenhuma bacia encontrada.")
            print(f"Bacias: {len(basins)}")

            print("Gerando Mapas...")
            figs = []
            maps_cfg = [
                ('Vazao_m3s', 'Hidrologia: Vazão Média', 'YlGnBu', 'm³/s'),
                ('Precipitacao_mm', 'Climatologia: Precipitação', 'Blues', 'mm/ano'),
                ('Floresta_pct', 'Uso do Solo: Cobertura Florestal', 'Greens', '%')
            ]
            
            for col, title, cmap, unit in maps_cfg:
                figs.append(self.renderer.generate_map(basins, aoi, col, title, cmap, unit))
            
            figs.append(self.renderer.generate_charts_page(basins))
            figs.extend(self.renderer.generate_table_pages(basins))
            
            self.cfg.PATH_OUTPUT.mkdir(parents=True, exist_ok=True)
            out = self.cfg.PATH_OUTPUT / f"RELATORIO_{GeoUtils.sanitize_filename(self.cfg.NOME_MUNICIPIO)}.pdf"
            
            with PdfPages(out) as pdf:
                for f in figs:
                    pdf.savefig(f, dpi=300, bbox_inches='tight')
                    plt.close(f)
            
            print(f"Relatório Salvo: {out}")

        except Exception as e:
            print(f"ERRO CRÍTICO: {e}")
            import traceback; traceback.print_exc()

if __name__ == "__main__":
    # Exemplo de Uso
    cfg = HydroConfig(NOME_MUNICIPIO='Três Corações', SIGLA_UF='MG')
    HydroPipeline(cfg).run()

--- INICIANDO: Três Corações ---
Consultando Earth Engine...
Bacias: 15
Gerando Mapas...
Relatório Salvo: ..\output\RELATORIO_tres_coracoes.pdf
